In [2]:
import json
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_colwidth', -1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [4]:
temp=open('/kaggle/input/convolve-epoch1/train.json','r')
file=json.load(temp)
df = pd.DataFrame(list(file.items()),columns=['log','lable'])

In [5]:
df=df.sample(frac=1,random_state=1).reset_index()
df.drop("index",axis=1,inplace=True)

In [6]:
x=df.drop('lable',axis=1)
y=df['lable']

In [7]:
import nltk
# train_token = x.apply(lambda row: nltk.word_tokenize(row['log']), axis=1)
# print(train_token)

In [8]:
y[y=='normal']=1

In [9]:
y[y=='abnormal']=0

In [10]:
voc_size=5000

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
df.log.replace("KERNEL","",regex=True,inplace=True)
df.log.replace("[^a-z' 'A-Z]","",regex=True,inplace=True)
df.log.replace("RAS","",regex=True,inplace=True)
df.log.replace("RMNFCJU RMNFCJU RAS KERNEL","",regex=True,inplace=True)
df.log.replace("RMNIJU  RMNIJU  KERNEL","",regex=True,inplace=True)
df.log.replace("   RMNIJU  RMNIJU   ","",regex=True,inplace=True)
df.log.replace("KERNMNTFFATAL","KERNMNTF FATAL",regex=True,inplace=True)
messages=x.copy()
import re
corpus = []
for i in range(0, len(messages)):
#     print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['log'][i])
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    corpus.append(review)


In [13]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 


In [14]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
# print(embedded_docs)

In [15]:
embedded_docs

array([[   0,    0,    0, ...,  559, 2633, 4268],
       [   0,    0,    0, ..., 3518, 1350, 1959],
       [   0,    0,    0, ..., 3518, 1350, 1959],
       ...,
       [   0,    0,    0, ..., 3518, 1350, 1959],
       [   0,    0,    0, ..., 2862, 2495,  217],
       [   0,    0,    0, ...,  757, 4995, 4873]], dtype=int32)

In [16]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# print(model.summary())


2022-12-20 20:59:29.787986: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 20:59:29.876725: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 20:59:29.877530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 20:59:29.878830: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [17]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final

In [ ]:
x=pd.DataFrame(X_final,columns )

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_test= np.asarray(y_test).astype(np.float32)

In [19]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=10000)

2022-12-20 20:59:34.341968: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 556456200 exceeds 10% of free system memory.
2022-12-20 20:59:34.963655: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 556456200 exceeds 10% of free system memory.
2022-12-20 20:59:35.425901: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-12-20 20:59:37.786076: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


279/279 [==============================] - 29s 92ms/step - loss: 0.0564 - accuracy: 0.9893 - val_loss: 6.4040e-04 - val_accuracy: 1.0000
Epoch 2/10
279/279 [==============================] - 25s 88ms/step - loss: 3.4205e-04 - accuracy: 1.0000 - val_loss: 1.8419e-04 - val_accuracy: 1.0000
Epoch 3/10
279/279 [==============================] - 24s 88ms/step - loss: 1.4218e-04 - accuracy: 1.0000 - val_loss: 1.0001e-04 - val_accuracy: 1.0000
Epoch 4/10
279/279 [==============================] - 25s 88ms/step - loss: 8.3848e-05 - accuracy: 1.0000 - val_loss: 6.4267e-05 - val_accuracy: 1.0000
Epoch 5/10
279/279 [==============================] - 25s 88ms/step - loss: 5.6736e-05 - accuracy: 1.0000 - val_loss: 4.4368e-05 - val_accuracy: 1.0000
Epoch 6/10
279/279 [==============================] - 25s 89ms/step - loss: 4.2519e-05 - accuracy: 1.0000 - val_loss: 3.0876e-05 - val_accuracy: 1.0000
Epoch 7/10
279/279 [==============================] - 25s 89ms/step - loss: 3.3010e-05 - accuracy: 1.00

In [20]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [21]:
test=pd.read_csv('/kaggle/input/convolve-epoch1/test.csv')

In [22]:
test.columns

Index(['ID', ' Log'], dtype='object')

In [23]:
test['Log'].replace("[^a-z' 'A-Z]","",regex=True,inplace=True)
test['Log'].replace("KERNEL","",regex=True,inplace=True)
test['Log'].replace("RAS","",regex=True,inplace=True)
test['Log'].replace("RMNFCJU RMNFCJU RAS KERNEL","",regex=True,inplace=True)

KeyError: 'Log'

In [ ]:

test['Log'].replace("RMNIJU  RMNIJU  KERNEL","",regex=True,inplace=True)
test['Log'].replace("   RMNIJU  RMNIJU   ","",regex=True,inplace=True)
test['Log'].replace("KERNMNTFFATAL","KERNMNTF FATAL",regex=True,inplace=True)

In [ ]:
test['Log']=test['Log'].apply(lambda x : x.lower())
x_test=test

In [ ]:
messages1=x_test.copy()
import re
corpus1 = []
for i in range(0, len(messages1)):
    review = re.sub('[^a-zA-Z]', ' ', messages1['Log'][i])
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    corpus1.append(review)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus1] 

In [ ]:
sent_length=50
embedded_docs1=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs1)

In [ ]:
X_test_final=np.array(embedded_docs1)

In [ ]:
y1=model.predict(X_test_final)

In [ ]:
prediction=pd.DataFrame(y1,columns = [' Label'])
print(type(prediction))

In [ ]:
print(prediction)

In [ ]:
prediction.to_csv('aslilmao.csv')

In [ ]:
patanhi=prediction.copy()


In [ ]:

for i in range(len(prediction)):
    if prediction[' Label'][i]> 0.0000000039:
        patanhi[' Label'][i]='normal'
    else:
        patanhi[' Label'][i]='anomaly'

In [ ]:
patanhi.to_csv('0.0000000038ka14.csv')